In [ ]:
!pip install snowflake-connector-python

In [ ]:
import snowflake.connector
import pandas as pd
con = snowflake.connector.connect(
    user='xxxx',
    password='xxxx',
    account='xxxx',
    warehouse='COMPUTE_WH',
    database='CREDITCARDFRAUD',
    schema='PUBLIC'
)

In [ ]:
query='select * from "BIKE"'
df=pd.read_sql(query,con)
con.close()
print(df.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
print(df.info())

In [ ]:
import numpy as np

# Cyclic encoding for HOUR
df['Hour_sin'] = np.sin(2 * np.pi * df['HOUR'] / 24)
df['Hour_cos'] = np.cos(2 * np.pi * df['HOUR'] / 24)


df['Hour_Temp'] = df['HOUR'] * df['TEMPERATURE']

# One-hot encoding of categorical columns
df = pd.get_dummies(df, columns=['SEASONS', 'HOLIDAY', 'FUNCTIONING_DAY'], drop_first=True)


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['RENTED_BIKE_COUNT', 'DATE'], axis=1)
y = df['RENTED_BIKE_COUNT']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
scaler = StandardScaler()
numeric_features = ['HOUR', 'TEMPERATURE', 'HUMIDITY', 'WIND_SPEED',
                    'VISIBILITY', 'DEW_POINT_TEMPERATURE', 'SOLAR_RADIATION',
                    'RAINFALL', 'SNOWFALL', 'Hour_sin', 'Hour_cos', 'Hour_Temp']

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])



In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)



In [ ]:

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")
